In [ ]:
# 1 #############################################################################
# %%capture is a cell magic command in Jupyter that captures the output (stdout,
# --- stderr, and display data) produced by the code within the cell.
#############################################################################
# It stores this output in a variable, allowing for further manipulation or
# --- suppression of the output.
#################################################################################
#################################################################################
# First install unsloth, xformers, trl, peft, accelerate, bitsandbytes, triton
#    --- and pytorch
#################################################################################
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes triton torch
!pip install xformers
#################################################################################

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-buckopd5/unsloth_32e8de5059f34bafaa650515c9c6cf13
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-buckopd5/unsloth_32e8de5059f34bafaa650515c9c6cf13
  Resolved https://github.com/unslothai/unsloth.git to commit 7a8f99e1890213cdd01a3ab6c3e13174a96e8220
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.

In [ ]:
# 2 #############################################################################
# We are set with the required libraries and functions that we need to fine-tune
# --- the desired LLM.
#################################################################################
# Import the necessary packages.
#################################################################################
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset, Dataset
from google.colab import files
from google.colab import drive
import pandas as pd
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from transformers import TextStreamer
import csv
import xformers
from transformers import TextStreamer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
import os
import random
import math
os.environ["llm_int8_enable_fp32_cpu_offload"] = "True"
os.environ["WANDB_API_KEY"] = "136c13053af3ebc23d01c1f8c4ff0bc2c3f6780f" # Replace with your actual API key
#################################################################################

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# 3 #############################################################################
# Setting some basic hyperparameter values
#################################################################################
# For max_seq_length the value can be any value since unsloth supports RoPE
# --- scaling internally
#################################################################################
# dtype can be set to None so that unsloth can auto detect the correct value
#   for it. (Float16 for Tesla T4, V100, Bfloat16 for Ampere+)
#################################################################################
# Setting load_in_4bit = True here makes sure that you only have to change the
# --- value of this parameter here rather than going to every other location
# --- that this parameter is used and changing each one of them one by one
#################################################################################
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [ ]:
# 4 #############################################################################
# fourbit_models:
# --- a machine learning model where the numerical values representing weights
# --- and activations within the network are stored and calculated using
# --- only 4 bits of information
#################################################################################
# We are using unsloth’s own 4bit quantized mistral-7B model because of GPU
# --- limitations in the Colab instance
##############################################################################
# these are the latest models available out of the box by unsloth
#################################################################################
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",

    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",

    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
]
#################################################################################

In [ ]:
# 5 #############################################################################
# FastLanguageModel.from_pretrained():
# --- a function within the Unsloth library that allows you to load a pre-trained
# --- large language model (LLM) quickly and efficiently, essentially providing
# --- a way to easily access and use a pre-trained model without having to train
# --- it from scratch.
#################################################################################
# We instantiate the model and it’s tokenizer
#################################################################################
# huggingface tokens can also be passed while instantiating the model, this is
# --- particularly helpful if we are trying to finetune gated models
#################################################################################
model, tokenizer = FastLanguageModel.from_pretrained(
    # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",

    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
#################################################################################

==((====))==  Unsloth 2025.4.1: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [ ]:
# 6 #############################################################################
# FastLanguageModel.get_peft_model():
# --- configure various parameters for finetuning, such as the number of
# --- attention heads, target modules, dropout rate, LoRa alpha and more.
#################################################################################
# Low-Rank matrix approximation:
# --- Instead of updating all the weights in a model, LoRA uses low-rank matrices
# --- to approximate the necessary changes,
# --- significantly reducing the number of parameters that need to be trained.
#################################################################################
# Frozen base model:
# --- When using LoRA, the majority of the pre-trained model's weights remain
# --- frozen, with only the newly added adapter matrices being updated during
# --- fine-tuning.
#################################################################################
# During fine-tuning, small, trainable matrices are inserted into selected layers
# --- of the pre-trained LLM.
#################################################################################
# These matrices are designed as a low-rank approximation of the changes needed
# --- to adapt the model to a new task.
#################################################################################
# Only the parameters within the LoRA adapters are updated during training,
# --- while the base model weights remain largely unchanged.
#################################################################################
# "get peft model" refers to loading a PEFT (Parameter-Efficient Fine-Tuning) model,
# --- which is a method for efficiently adapting pre-trained language models
# --- to specific tasks by fine-tuning only a small portion of the model's parameters,
# --- rather than the entire model, as explained by Hugging Face.
#################################################################################
# we take the peft version of our model
#################################################################################
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
# model = model.quantize(4)
#################################################################################

Unsloth 2025.4.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
# 7 #############################################################################
# We have to define a formatting function to format the sentences which is to
# --- be sent to the model
#################################################################################
# See: https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1/
#################################################################################
prompt = """Below given is an input comment that needs to be converted to a vague but correct target comment.
### Input Comment:
{}

### Target Comment:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["input"]
    targets      = examples["target"]
    texts = []
    for input, target in zip(inputs, targets):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(input, target) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
#################################################################################

In [ ]:
# 8 #############################################################################
# Data can be prepared in any way that the user sees fit
#################################################################################
# I have decided to read from a csv file and then convert the csv to a dictionary,
# --- so that there will be two values: Keys, Values
#################################################################################
# The csv file is too large, so we will only train on SAMPLE_SIZE records.
#################################################################################
data_dict = {}
SAMPLE_SIZE = 10_000
count = 0
#################################################################################
# The user (me) must be watchful here, since permission must be given to access
# --- the csv file on the Google account!!!
#################################################################################
drive.mount('/content/drive')
with open('/content/drive/My Drive/Colab Notebooks/youtube_comments_1.csv', mode='r', newline='') as input_file:
    csv_reader = csv.reader(input_file)
    data_dict["input"] = []
    data_dict["target"] = []
    ###########################################
    for items in csv_reader:
      count += 1
      if count > SAMPLE_SIZE:
        break
    ###########################################
      data_dict["input"].append(items[0])
      data_dict["target"].append(items[1])
###############################################
# CommentOut OR UnComment To See Data Being Loaded!!!
print (data_dict["input"])
print (data_dict["target"])
#################################################################################

Mounted at /content/drive
['Anyone know what movie this is?', "The fact they're holding each other back while equally being most aggressive 😂😂", 'waiting next video will be?', "Thanks for the great video.\n\nI don't understand why the DB continues to be accesible through port 8080 when the local machine connects to the docker container through port 5432? Or, why is it not possible to make GET requests to port 5432 on my local machine?\n\nIs it just because the port 5432 is only the port used to connect the app to the database, but the app itself is still handling HTTP through 8080? So an incoming request hits 8080, then goes through 5432 to get to the database?", 'Good person helping good people.\nThis is how it is in America with the exception of NY and DC.', 'Dei løk de seim😂', 'Number two because it looks the best with it', 'Thank God we don’t have to listen to his drivel anymore', 'Very similar thing happened to me! We lived near a beach that had a stray dog. We would all leave foo

In [ ]:
# 9 #############################################################################
# A DataSet is created from the Python Dictionary, to be used at the next training
# --- step.
#################################################################################
dataset = Dataset.from_dict(data_dict, )
dataset = dataset.map(formatting_prompts_func, batched=True)
# CommentOut Or UnComment As Appropriate to see the DataSet contents!!!
# print (dataset['text'])
print (dataset['input'])
print (dataset['target'])
#################################################################################

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

['Anyone know what movie this is?', "The fact they're holding each other back while equally being most aggressive 😂😂", 'waiting next video will be?', "Thanks for the great video.\n\nI don't understand why the DB continues to be accesible through port 8080 when the local machine connects to the docker container through port 5432? Or, why is it not possible to make GET requests to port 5432 on my local machine?\n\nIs it just because the port 5432 is only the port used to connect the app to the database, but the app itself is still handling HTTP through 8080? So an incoming request hits 8080, then goes through 5432 to get to the database?", 'Good person helping good people.\nThis is how it is in America with the exception of NY and DC.', 'Dei løk de seim😂', 'Number two because it looks the best with it', 'Thank God we don’t have to listen to his drivel anymore', 'Very similar thing happened to me! We lived near a beach that had a stray dog. We would all leave food out for her. One day on 

In [ ]:
# 10 #############################################################################
# We define the trainier, we will be using the SFTTrainer ( Supervised Finetuning )
#    available in the transformers library.
##################################################################################
# Unsloth also supports the DPOTrainer.
#################################################################################
# I found pretty good results by setting max_steps = 10 below and SAMPLE_SIZE = 10_000
# --- above!!!
#################################################################################
# After creating the Supervised Fine Tuning Trainer, it is called and training
# --- begins!!!
#################################################################################
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        max_steps = 10, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        # output_dir = '/content/drive/My Drive/Colab Notebooks/output.txt',
    ),
)

trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dumaharacka (dumaharacka-university-of-tennessee-chattanooga) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,4.541800
2,4.657400
3,4.114000
4,3.419800
5,2.885900
6,2.294400
7,2.084800
8,1.737300
9,1.593300
10,1.370400


Unsloth: Will smartly offload gradients to save VRAM!


In [18]:
# 11 ############################################################################
# The model is now trained, so we may use it to determine if a YouTube comment
# --- is Positive, Neutral, or Negative.
#################################################################################
# One may change the statement in the quotes to which of the designated values
# -- the statement will be assigned!!!
#################################################################################
streamer =TextStreamer(tokenizer)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    prompt.format(
        "The scene was okay.", # input
        "" #target
    )
], return_tensors = "pt").to("cuda")

_ = model.generate(**inputs, streamer=streamer, max_new_tokens = 256, use_cache = True)
#################################################################################

<s> Below given is an input comment that needs to be converted to a vague but correct target comment.
### Input Comment:
The scene was okay.

### Target Comment:

Neutral
</s>
